In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import numpy as np
import torch
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd

In [5]:
sys.path.insert(
    0, os.path.abspath(os.path.join(os.getcwd(), "../../DermSynth3D_private"))
)
from dermsynth3d.models.model import SkinDeepLabV3
from dermsynth3d.datasets.datasets import (
    Fitz17KAnnotations,
    ImageDataset,
    Ph2Dataset,
)
from dermsynth3d.utils.colorconstancy import shade_of_gray_cc
from scripts.inference import inference_multitask
from dermsynth3d.losses.metrics import (
    compute_results,
    conf_mat_cells,
)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
multitask_model = SkinDeepLabV3(multi_head=False, freeze_backbone=False)
multitask_model.load_state_dict(torch.load("/path/to/trained/model"), strict=False)

<All keys matched successfully>

In [8]:
multitask_model = multitask_model.to(device)
multitask_model.eval();

In [9]:
img_size = (320, 320)

In [10]:
# Assumes the model was pretrained using these values.
preprocess_input = A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
img_preprocess = A.Compose(
    [
        preprocess_input,
    ]
)

# To force a resize of the input image.
resize_func = A.Resize(
    height=img_size[0], width=img_size[1], interpolation=cv2.INTER_CUBIC
)
# Perform spatial augmentation on both the image and mask.
spatial_augment = A.Compose(
    [
        resize_func,
    ]
)
resize_aspect_smallest = A.augmentations.geometric.resize.SmallestMaxSize(
    max_size=img_size[0], always_apply=True
)

<h2>Fitzpatrick 17k test annotations</h2>

https://github.com/mattgroh/fitzpatrick17k

See instructions to download the images.

Update `dir_fitz_images` to the directory containing the fitzpatrick17k images.

Update `dir_fitz_test_annotations` to the subset of masks provided.

In [17]:
# Where the fitzpatrick images are stored.
dir_fitz_images = "../data/fitzpatrick17k/data/finalfitz17k/"
# Where the manual annotations are stored.
dir_fitz_test_annotations = "../data/fitzpatrick17k/annotations/annotations/"

fitz_test_ds = Fitz17KAnnotations(
    dir_images=dir_fitz_images,
    dir_targets=dir_fitz_test_annotations,
    image_extension=".jpg",
    target_extension=".png",
    spatial_transform=resize_aspect_smallest,
    image_augment=None,
    image_preprocess=img_preprocess,
    totensor=ToTensorV2(transpose_mask=True),
    color_constancy=shade_of_gray_cc,
)
fitz_test_dataloader = DataLoader(fitz_test_ds, batch_size=1, shuffle=False)
len(fitz_test_dataloader)

25

In [20]:
dir_skin_preds_fitz17k_test = "../data/fitzpatrick17k/annotations/test/skin/"

fitz_test = inference_multitask(
    max_imgs=len(fitz_test_dataloader),
    model=multitask_model,
    dataloader=fitz_test_dataloader,
    device=device,
    save_to_disk=True,
    return_values=True,
    dir_anatomy_preds="../data/fitzpatrick17k/annotations/test/anatomy/",
    dir_save_images="../data/fitzpatrick17k/annotations/test/images/",
    dir_save_skin_preds=dir_skin_preds_fitz17k_test,
)

  0%|          | 0/25 [00:00<?, ?it/s]

In [41]:
fitz_test_df = pd.DataFrame(
    compute_results(fitz_test_ds, dir_skin_preds_fitz17k_test, pred_ext=".png")
)

# Skin condition.
print(
    "{:.2f} \pm {:.2f}".format(
        fitz_test_df.lesion_ji.mean(),
        fitz_test_df.lesion_ji.std(),
    )
)

# Skin.
print(
    "{:.2f} \pm {:.2f}".format(
        fitz_test_df.skin_ji.mean(),
        fitz_test_df.skin_ji.std(),
    )
)

# Nonskin.
print(
    "{:.2f} \pm {:.2f}".format(
        fitz_test_df.nonskin_ji.mean(),
        fitz_test_df.nonskin_ji.std(),
    )
)

0.61 \pm 0.23
0.88 \pm 0.10
0.60 \pm 0.43


<h2>PH2 results</h2>

https://www.fc.up.pt/addi/ph2%20database.html

Update `dir_images` to your path to the images.

In [35]:
resize_aspect_longest = A.augmentations.geometric.resize.LongestMaxSize(
    max_size=img_size[0],always_apply=True)

ph2_img_augment = A.Compose([
    A.GaussianBlur(blur_limit=(9,9), always_apply=True),
])

ph2_ds = Ph2Dataset(
    dir_images='../data/ph2/images'
    dir_targets=None,
    name='ph2',
    image_extension='.bmp', 
    target_extension='.bmp',
    image_augment=ph2_img_augment,
    spatial_transform=resize_aspect_longest,
    image_preprocess=img_preprocess,
    totensor=ToTensorV2(transpose_mask=True),
    color_constancy=shade_of_gray_cc,
)
print(len(ph2_ds))
ph2_dataloader = DataLoader(ph2_ds, batch_size=1, shuffle=False)

200


In [37]:
dir_skin_preds_ph2_test = "../data/ph2/preds/skin/"

ph2_preds = inference_multitask(
    max_imgs=len(ph2_dataloader),
    model=multitask_model,
    dataloader=ph2_dataloader,
    device=device,
    save_to_disk=True,
    return_values=False,
    dir_save_images="../data/ph2/preds/images/",
    dir_save_targets="../data/ph2/preds/targets/",
    dir_save_skin_preds=dir_skin_preds_ph2_test,
)

  0%|          | 0/200 [00:00<?, ?it/s]

In [40]:
ph2_test_df = pd.DataFrame(
    compute_results(ph2_ds, dir_skin_preds_ph2_test, pred_ext=".bmp")
)

# Skin condition.
print(
    "{:.2f} \pm {:.2f}".format(
        ph2_test_df.lesion_ji.mean(),
        ph2_test_df.lesion_ji.std(),
    )
)

0.62 \pm 0.21


<h2>Dermofit results</h2>

https://licensing.edinburgh-innovations.ed.ac.uk/product/dermofit-image-library

Requires a license and a fee.

Update `dir_images` and `dir_targets` to your paths.

In [42]:
dermofit_img_augment = A.Compose(
    [
        A.GaussianBlur(blur_limit=(3, 3), always_apply=True),
    ]
)
dermofit_ds = ImageDataset(
    dir_images="../data/dermofit/images/",
    dir_targets="../data/dermofit/targets/",
    name="dermofit",
    image_extension=".png",
    target_extension=".png",
    image_augment=None,
    spatial_transform=resize_aspect_smallest,
    image_preprocess=img_preprocess,
    totensor=ToTensorV2(transpose_mask=True),
    color_constancy=shade_of_gray_cc,
)
print(len(dermofit_ds))
dermofit_dataloader = DataLoader(dermofit_ds, batch_size=1, shuffle=False)

1300


In [43]:
dir_skin_preds_dermofit = "../data/dermofit/predictions/"

dermofit_preds = inference_multitask(
    max_imgs=len(dermofit_dataloader),
    model=multitask_model,
    dataloader=dermofit_dataloader,
    device=device,
    save_to_disk=True,
    return_values=False,
    dir_save_skin_preds=dir_skin_preds_dermofit,
)

  0%|          | 0/1300 [00:00<?, ?it/s]

In [44]:
dermofit_df = pd.DataFrame(
    compute_results(dermofit_ds, dir_skin_preds_dermofit, pred_ext=".png")
)

# Skin condition.
print(
    "{:.2f} \pm {:.2f}".format(
        dermofit_df.lesion_ji.mean(),
        dermofit_df.lesion_ji.std(),
    )
)

0.57 \pm 0.21


<h2>Pratheepan results on FacePhoto</h2>

Data from:
https://web.fsktm.um.edu.my/~cschan/downloads_skin_dataset.html

Update `dir_images` and `dir_targets` to your data paths to the images and skin masks.

In [52]:
pratheepan_ds = ImageDataset(
    dir_images="../data/Pratheepan_Dataset/FacePhoto/",
    dir_targets="../data/Pratheepan_Dataset/GroundT_FacePhoto/",
    name="pratheepan",
    image_extension=".jpg",
    target_extension=".png",
    image_augment=None,
    spatial_transform=resize_aspect_smallest,
    image_preprocess=img_preprocess,
    totensor=ToTensorV2(transpose_mask=True),
    color_constancy=shade_of_gray_cc,
)

In [55]:
dir_skin_preds_pratheepan = "../data/Pratheepan_Dataset/predictions/skin/"

pratheepan_dataloader = DataLoader(pratheepan_ds, batch_size=1, shuffle=False)
pratheepan_preds = inference_multitask(
    max_imgs=len(pratheepan_ds),
    model=multitask_model,
    dataloader=pratheepan_dataloader,
    device=device,
    save_to_disk=True,
    return_values=False,
    dir_anatomy_preds="../data/Pratheepan_Dataset/predictions/anatomy/",
    dir_save_skin_preds=dir_skin_preds_pratheepan,
)

  0%|          | 0/32 [00:00<?, ?it/s]

In [57]:
pratheepan_df = pd.DataFrame(
    compute_results(pratheepan_ds, dir_skin_preds_pratheepan, pred_ext=".png")
)

# Skin condition.
print(
    "{:.2f} \pm {:.2f}".format(
        pratheepan_df.skin_ji.mean(),
        pratheepan_df.skin_ji.std(),
    )
)

0.76 \pm 0.14


In [64]:
res = conf_mat_cells(pratheepan_ds, dir_skin_preds_pratheepan, ".png")
tps = res["tps"]
fps = res["fps"]
fns = res["fns"]

In [65]:
f1 = (2 * np.sum(tps)) / ((2 * np.sum(tps)) + np.sum(fps) + np.sum(fns))
f1

0.8615441543762365